# 📘 Example: Database Builder

In this notebook we are going to create a complete **database** for our FloodAdapt model in Charleston, USA. 
To do so we use the in FloodAdapt integrated **Database builder**, which allows for a quick and easy database creation!
We presume that you have already created a functioning [**Delft-FIAT**](https://github.com/Deltares/Delft-FIAT) and [**SFINCS model**](https://github.com/Deltares/SFINCS) and we"ll go from there. 

To build a FloodAdapt database we need to set up the **configurations** for the **DatabaseBuilder**. The configuration file consists of:
1. **Basic model parameters** to create a simple FloodAdapt database.
2. **Advance model parameters** to create a more complex FloodAdapt database.

The configuration file can be either created through the FloodAdapt objects or can we parsed as a simple dictionary.
We advice you to work with the FloodAdapt classes, unless your database only has a few simple inputs. In this notebook we will demonstrate both options for a simple and an advanced FloodAdapt database.

## Import libraries

In [ ]:
# Import packages
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from shapely.geometry import Polygon
import toml

import flood_adapt.adapter.fiat_adapter as fiat
import flood_adapt.database_builder.database_builder as db
from flood_adapt.objects.forcing.tide_gauge import TideGaugeSource
from flood_adapt.objects.forcing.timeseries import Scstype
from flood_adapt import unit_system as us
%matplotlib inline

## 🔍 **Step 1**. Explore the Delft-FIAT model

The study area is in **Charleston, USA**, a coastal city on the East Coast of the United States. 
So, let's first inspect the exposure data in the **Delft-FIAT model** to get an understanding of our study area.  

We initiate the **FiatAdapter** in FloodAdapt with the model root of the Delft-FIAT model and explore the exposure data in the map.

In [ ]:
# Initiate the FiatAdapter class from FloodAdapt and plot the Fiat model. 
# Set up the settings for the database
# Define the static data folder
STATIC_DATA_DIR = Path("../_data/examples/static-data/1_DBBuilder").resolve()
fn_fiat = Path(STATIC_DATA_DIR  / "fiat") 
fa = fiat.FiatAdapter(
    model_root =  fn_fiat
    )

# Get the geodataframe with exposure data
gdf = fa._model.exposure.get_full_gdf(fa._model.exposure.exposure_db)

# Plot the region and the secondary_object_types of the exposure data
m = gdf.explore(column="secondary_object_type", name="Exposure types")
m

## 📄 **Step 2**: Build a **simple** FloodAdapt model

In this step we will look at the basic model parameters to create a **simple** FloodAdapt model.  
You can either create the configuration file manually with a dictionary or use the integrated classes in FloodAdapt for a more streamlined approach. 
We show you both options below.

### 📚 **Step 2a**: Build the configuration from the **Database Builder classes**

To create the configuration object we need to initiate the `db.ConfigModel` object.
After we define some general attributes we need to set the GUI parameters in the `gui` attribute. The `gui` attributs define how the output is visualized in the GUI. More importantly we need to point to the Delft-FIAT and SFINCS model by providing the path in form of a `str` to the `fiat` and `sfincs_overland` attributes, respectively.

**Important!**  <span style="color:red;">double check</span>
The water level reference should be set to the reference of your DEM. You can create a reference system manually as shown below or fetch this information from a close-by tide gauge. This will be shown in the more advanced options. 

In [ ]:

# Initiate the config_model for a simple FloodAdapt Database configuration file 
config_model = db.ConfigModel(
    name="charleston_example3",
    database_path= str((STATIC_DATA_DIR /"output"/"Database").absolute()),
    unit_system= db.UnitSystems.imperial,
    gui=db.GuiConfigModel(
        max_flood_depth=5,
        max_aggr_dmg=1e6,
        max_footprint_dmg=250000,
        max_benefits=5e6,
    ),
    sfincs_overland=db.FloodModel(
        name=str((STATIC_DATA_DIR  / "sfincs").absolute()),
        reference="MSL",
    ),
    fiat=str((STATIC_DATA_DIR / "fiat").absolute()),
)

### 📖 **Step 2b**: Create a configuration file from a **dictionary**
We can create a simple dictionary with all the attributes and save it to a configuration file.

In [ ]:
# Create the configuration file for the database builder for a simple FloodAdapt database. 
# all paths should be provided with forward slashes (/)
config = {
"name": "charleston_example1",
"database_path": "Database",
"sfincs_overland" : { 
    "name":"sfincs",
    "reference":"MSL"
    },
"fiat" : "fiat",
"unit_system" :"imperial",
"gui": {
"max_flood_depth": 5,
"max_aggr_dmg" : 10000000,
"max_footprint_dmg": 250000,
"max_benefits" : 50000000}
}

When we work with a dictionary, we need to save the configuration file as a .toml-file, so that we can call it later when we run the Database Builder.

In [ ]:
# Save the configuration file
with open(STATIC_DATA_DIR /"db_config.toml", "w") as f:
    toml.dump(config, f)

## 🏃‍♀️ **Step 3**: Run the Database Builder

We are ready to run the Database Builder with the configuration that we just created above. First we are going to run **Option 2a** - in which we generated the configuration using the FloodAdapt classes.

In [ ]:
# Run Option 2a -  DB-builder config from FloodAdapt classes
db_build = db.DatabaseBuilder(config_model)
db_build.build(overwrite= True)

Here we are going to run **Option 2b** - in which we manually created a configuration file from a dictionary. 

In [ ]:
# Run Option 2b -  DB-builder config from dictionary
config_path = STATIC_DATA_DIR  / "db_config.toml"
config = db.ConfigModel.read(config_path)
dbs = db.DatabaseBuilder(config)
dbs.build(overwrite = True)

Now you created two complete FloodAdapt Database. Both databases should be identical as we used the same inputs. You can open the databases in the GUI and explore them further or continue working with the database through the API. 

## 🗂️ **Step 4**. Build an **advanced** FloodAdapt model

In the previous step we created a simple FloodAdapt model. In this ex step we look at the more advanced configurations and will create a comprehensive FloodAdapt model. 

The configuration file consists of:
1. **Basic model parameters** to create a simple FloodAdapt database.
2. **Advance model parameters** to create a more complex FloodAdapt database.

Additionally, to the basic model parameters (see step 2), you have the option for the following additions to create a more complex model:  
- `probabilistic_set`: The file path to a probabilistic dataset
- `infographics`: Include infographics in output. Boolean Tue/ False
- `return_periods`: A list of return periods. Integers or Floats

*Sfincs*

- `sfincs_offshore`:  A dictinary of the offshore sfincs model with the entries: file path, reference system (eg. "MSL") 
- `dem`: A dictinary of DEM with the entries: file path, units
- `excluded_datums`: A list of datums <span style="color:red;">double check</span>
- `scs`: A dictinary of the SCS (Soil Conservation Service) rainfall with the entries: file path, scs type
- `tide_gauge`: A dictinary of tide gauges with the entries: source, max_distance(dict: values, units)
- `cyclones`: Boolean (True/False)
- `cyclone_basin`:  <span style="color:red;">double check</span>
- `obs_point`: A list of dictionaries with the entries: name, description, lat, lon

*FIAT*

- `aggregation_areas`: A list of dictionaries with the entries: name, file path, field_name. Aggregates the exposure into larger spatial groups o summarize impacts on larger scale. 
- `building_footprints`:  <span style="color:red;">double check</span>
- `fiat_buildings_name`: The name of the exposure type for buildings. Default is set to **"buildings"**
- `fiat_roads_name`:  The name of the exposure type for roads. Default is set to **"roads"**
- `bfe`: The base flood elevation model. A list of dictionaries with the entries: name, file path, field_name.
- `svi`: The social vulnerability index. A dictionary with the entries: file path, field_name, threshold
- `road_width`: The width of the road. Integer or Float

*NOTE!* These parameers are **additional** to the basic parameters, so don"t forge to add these to your configuration as well!

### **Step 4a**. Create a configuration file from the Database Builder classes

In [ ]:
config_model = db.ConfigModel(
    name="charleston_example_advanced1",
    database_path= str((STATIC_DATA_DIR /"Database").absolute()),
    unit_system= db.UnitSystems.imperial,
    gui=db.GuiConfigModel(
        max_flood_depth=5,
        max_aggr_dmg=1e6,
        max_footprint_dmg=250000,
        max_benefits=5e6,
    ),
    infographics=True,
    probabilistic_set=str(Path(STATIC_DATA_DIR  / "test_set")), #TODO fix final path
    return_periods=[1, 2, 5, 10, 25, 50, 100],
    sfincs_overland=db.FloodModel(
        name=str((STATIC_DATA_DIR  / "sfincs").absolute()),
        reference="MSL",
    ),
    sfincs_offshore=db.FloodModel(
        name=str(Path(STATIC_DATA_DIR / "offshore")),  #TODO fix final path
        reference="MSL",
        vertical_offset=us.UnitfulLength(
            value=0.33, units=us.UnitTypesLength.feet
        )),
    dem=db.DemModel(
        filename=str(Path(STATIC_DATA_DIR / "charleston_14m.tif")),  #TODO fix final path
        units=us.UnitTypesLength.meters,
    ),
    excluded_datums=["NAVD88"],
    #references=db.WaterlevelReferenceModel(
    #    reference="MSL",
    #    datums=[
    #        db.DatumModel(name="MSL", height=us.UnitfulLength(value=0, units=us.UnitTypesLength.meters)),
    #        db.DatumModel(name="NAVD88", height=us.UnitfulLength(value=1, units=us.UnitTypesLength.meters))
    #    ]
    #),
    slr_scenarios=db.SlrScenariosModel(
        file=str(Path(STATIC_DATA_DIR  / "slr.csv")),
        relative_to_year=2020,
    ),
    scs=db.SCSModel(
        file=str(Path(STATIC_DATA_DIR / "scs_rainfall.csv")),
        type=Scstype.type3,
    ),
    tide_gauge=db.TideGaugeConfigModel(
        source=db.TideGaugeSource.noaa_coops,
        max_distance=us.UnitfulLength(
            value=100, units=us.UnitTypesLength.miles
        ),
    ),
    cyclones=True,
    cyclone_basin=db.Basins.NA,
    obs_point=[
        db.ObsPointModel(
            name="ashley_river",
            description="Ashley River - James Island Expy",
            lat=32.7765,
            lon=-79.9543,
        ),
        db.ObsPointModel(
            name=8665530,
            description="Charleston Cooper River Entrance",
            ID=8665530,
            lat=32.78,
            lon=-79.9233,
        ),
    ],
    fiat=str(Path(STATIC_DATA_DIR / "fiat").absolute()),
    #aggregation_areas=[
    #db.SpatialJoinModel(
    #    name="aggr_lvl_1",
    #    file=str(
    #        Path(STATIC_DATA_DIR / aggr_lvl_1.geojson")
    #    ),
    #    field_name="name",
    #),
    #db.SpatialJoinModel(
    #    name="aggr_lvl_2",
    #    file=str(Path(STATIC_DATA_DIR /aggr_lvl_2.geojson")
    #    ),
    #    field_name="name",
    #),
#],
    building_footprints=db.FootprintsOptions.OSM,
    fiat_buildings_name="buildings",
    fiat_roads_name="roads",
    bfe=db.SpatialJoinModel(
        file=str(Path(STATIC_DATA_DIR / "bfe.geojson")),
        name="bfe",
        field_name="bfe",
    ),
    svi=db.SviConfigModel(
        file=str(Path(STATIC_DATA_DIR / "CDC_svi_2020.gpkg")),
        field_name="SVI",
        threshold=0.5,
    ),
    road_width=5,
    )

### **Step 4b**. Create a configuration file from a dictionary

In [ ]:
# Create the configuration file for the database builder for a simple FloodAdapt database. 

# all paths should be provided with forward slashes (/)
config = {"name": "charleston_example_advanced2",
 "database_path": Path(STATIC_DATA_DIR / "Database"),
 "unit_system": "imperial",
 "gui": {"max_flood_depth": 5.0,
  "max_aggr_dmg": 1000000.0,
  "max_footprint_dmg": 250000.0,
  "max_benefits": 5000000.0},
 "infographics": True,
 "fiat": Path(STATIC_DATA_DIR / "fiat"),
 #"aggregation_areas": [{"name": "aggr_lvl_1",
 #  "file": Path(STATIC_DATA_DIR / "aggr_lvl_1.geojson"),
 #  "field_name": "name"},
 # {"name": "aggr_lvl_2",
 #  "file": Path(STATIC_DATA_DIR / "aggr_lvl_2.geojson"),
 #  "field_name": "name"}],
 "building_footprints": "OSM",
 "fiat_buildings_name": "buildings",
 "fiat_roads_name": "roads",
 "bfe": {"name": "bfe",
  "file": Path(STATIC_DATA_DIR / "bfe.geojson"),
  "field_name": "bfe"},
 "svi": {"name": None,
  "file": Path(STATIC_DATA_DIR / "CDC_svi_2020.gpkg"),
  "field_name": "SVI",
  "threshold": 0.5},
 "road_width": 5.0,
 "return_periods": [1, 2, 5, 10, 25, 50, 100],
 "sfincs_overland": {"name": Path(STATIC_DATA_DIR / "sfincs"),
  "reference": "MSL",
  "vertical_offset": None},
 "sfincs_offshore": {"name": Path(STATIC_DATA_DIR / "offshore"),
  "reference": "MSL",
  "vertical_offset": {"value": 0.33, "units":"feet"}},
 "dem": {"filename": Path(STATIC_DATA_DIR / "charleston_14m.tif"),
  "units": "meters"},
 "excluded_datums": ["NAVD88"],
 "slr_scenarios": {"file": Path(STATIC_DATA_DIR / "slr.csv"),
  "relative_to_year": 2020},
 "scs": {"file": Path(STATIC_DATA_DIR / "scs_rainfall.csv"),
  "type": "type_3"},
 "tide_gauge": {"source":"noaa_coops",
  "description": "",
  "max_distance": {"value": 100.0, "units": "miles"}},
 "cyclones": True,
 "cyclone_basin": "NA",
 "obs_point": [{"name": "ashley_river",
   "description": "Ashley River - James Island Expy",
   "lat": 32.7765,
   "lon": -79.9543},
  {"name": 8665530,
   "description": "Charleston Cooper River Entrance",
   "ID": 8665530,
   "lat": 32.78,
   "lon": -79.9233}],
 "probabilistic_set": Path(STATIC_DATA_DIR / "test_set")}

# Save the configuration file
with open(Path(STATIC_DATA_DIR /"output"/"db_config_advanced.toml", "w")) as f:
    toml.dump(config, f)

## 🏃‍♀️ **Step 5**. Run the Database Builder

We are ready to run the Database Builder with the configuration that we just created above. First we are going to run **Option 3a** - in which we generated the configuration using the FloodAdapt classes. After we are going to run **Option 3b** - in which we manually created a configuration file from a dictionary. 

In [ ]:
# Run Option 2a -  DB-builder config from FloodAdapt classes
db_build = db.DatabaseBuilder(config_model)
db_build.build(overwrite= True)

# Run Option 2b -  DB-builder config from dictionary
config_path = Path(STATIC_DATA_DIR / "output" / "db_config_advanced.toml")
config = db.ConfigModel.read(config_path)
dbs = db.DatabaseBuilder(config)
dbs.build(overwrite = True)

Now you created a complete and advanced FloodAdapt Database. You can open the databases in the GUI and explore them further. 

<div>
<img src="../_static/images/example_chaleston2.png" width="900"/>
</div>

## **Finished!**